In [52]:
import os
import shutil


import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
import datetime
import math
import warnings

import numpy as np
import pandas as pd
import prophet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.models import Sequential


warnings.filterwarnings("ignore")

### Univariate LSTM models

In [53]:
#Pour mes datas, j'importe, je choisis le dataset avec les revenus et la date d'ConnectionAbortedError
file = 'tgdata.csv'
df= pd.read_csv(file)
df.head(5)

,Unnamed: 0,dt,subs,revenu_data_pack,BData_1K_700M_subs,BData_200_120M_subs,BData_200_90M_subs,BData_350_170M_subs,BData_350_220M_subs,BData_5000_4G_subs,...,WelcomePack_revenus,XTRANET100_revenus,XTRANET1000_revenus,XTRANET1500_revenus,XTRANET200_revenus,XTRANET3000_revenus,XTRANET350_revenus,XTRANET50_revenus,XTRANET600_revenus,XTRANuit_revenus
0,0,2022-01-01,440,166850.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2022-01-02,370,134750.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2022-01-03,368,129300.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,2022-01-04,351,135850.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,2022-01-05,361,134350.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
df = df.drop('Unnamed: 0', axis=1)
df.head(5)

,dt,subs,revenu_data_pack,BData_1K_700M_subs,BData_200_120M_subs,BData_200_90M_subs,BData_350_170M_subs,BData_350_220M_subs,BData_5000_4G_subs,BData_600_500M_subs,...,WelcomePack_revenus,XTRANET100_revenus,XTRANET1000_revenus,XTRANET1500_revenus,XTRANET200_revenus,XTRANET3000_revenus,XTRANET350_revenus,XTRANET50_revenus,XTRANET600_revenus,XTRANuit_revenus
0,2022-01-01,440,166850.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2022-01-02,370,134750.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2022-01-03,368,129300.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2022-01-04,351,135850.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2022-01-05,361,134350.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
data_time = df[['dt', 'revenu_data_pack']]
data = df[['revenu_data_pack']]
print(data.shape)
data.head(5)


(1002, 1)


,revenu_data_pack
0,166850.0
1,134750.0
2,129300.0
3,135850.0
4,134350.0


Nous avons donc nos données pour le traitement supervisé univarié.

In [56]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


In [57]:
# transform univariate time series to supervised learning problem
from numpy import array
 
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)




In [58]:
# define input sequence
raw_seq = list(data['revenu_data_pack'])
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

[166850. 134750. 129300.] 135850.0
[134750. 129300. 135850.] 134350.0
[129300. 135850. 134350.] 125100.0
[135850. 134350. 125100.] 152050.0
[134350. 125100. 152050.] 126150.0
[125100. 152050. 126150.] 146800.0
[152050. 126150. 146800.] 147900.0
[126150. 146800. 147900.] 123450.0
[146800. 147900. 123450.] 118050.0
[147900. 123450. 118050.] 138350.0
[123450. 118050. 138350.] 126150.0
[118050. 138350. 126150.] 139100.0
[138350. 126150. 139100.] 122350.0
[126150. 139100. 122350.] 116250.0
[139100. 122350. 116250.] 126950.0
[122350. 116250. 126950.] 120900.0
[116250. 126950. 120900.] 156650.0
[126950. 120900. 156650.] 109650.0
[120900. 156650. 109650.] 139350.0
[156650. 109650. 139350.] 180800.0
[109650. 139350. 180800.] 117550.0
[139350. 180800. 117550.] 116050.0
[180800. 117550. 116050.] 116050.0
[117550. 116050. 116050.] 116700.0
[116050. 116050. 116700.] 163300.0
[116050. 116700. 163300.] 112500.0
[116700. 163300. 112500.] 140750.0
[163300. 112500. 140750.] 107750.0
[112500. 140750. 107

In [59]:
#preparer les données pour LSTM
# transform input from [samples, features] to [samples, timesteps, features]
n_features = 1 
X = X.reshape((X.shape[0], X.shape[1], 1))
print(X.shape)


(999, 3, 1)


### Vanilla LSTM

In [60]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [61]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [62]:
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [63]:
# demonstrate prediction
x_input = array([147995,  99996, 137198])
x_input = x_input.reshape((1, n_steps, n_features))

In [64]:
yhat = model.predict(x_input, verbose=0)
yhat

array([[125498.39]], dtype=float32)

### Stacked LSTM

Multiple hidden LSTM layers can be stacked one on top of another in what is referred to as
a Stacked LSTM model. An LSTM layer requires a three-dimensional input and LSTMs by
default will produce a two-dimensional output as an interpretation from the end of the sequence.
We can address this by having the LSTM output a value for each time step in the input data by
setting the return
sequences=True argument on the layer. This allows us to have 3D output
from hidden LSTM layer as input to the next. We can therefore de ne a Stacked LSTM as
follows.

In [65]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps,
n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [66]:
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = array([147995,  99996, 137198])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[124983.18]]


### Bidirectionnal LSTM


Pour certains problèmes de prédiction de séquences, il peut être avantageux de permettre au modèle LSTM d'apprendre la séquence d'entrée à la fois dans le sens direct et dans le sens inverse, puis de combiner ces deux interprétations. Ce type de modèle est appelé un LSTM bidirectionnel. On peut implémenter un LSTM bidirectionnel pour la prévision de séries temporelles univariées en enveloppant la première couche cachée dans une couche spéciale appelée Bidirectional.

Cela permet au modèle de lire l'entrée dans les deux directions, optimisant ainsi sa compréhension des dépendances dans les données. Voici un exemple de définition d'un LSTM bidirectionnel qui traite l'entrée dans les deux sens.

In [67]:
# define model
model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = array([147995,  99996, 137198])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[127257.266]]


### CNN LSTM

A convolutional neural network, or CNN for short, is a type of neural network developed for
working with two-dimensional image data. The CNN can be very e ective at automatically
extracting and learning features from one-dimensional sequence data such as univariate time
series data. A CNN model can be used in a hybrid model with an LSTM backend where the
CNN is used to interpret subsequences of input that together are provided as a sequence to an
LSTM model to interpret. This hybrid model is called a CNN-LSTM.
The rst step is to split the input sequences into subsequences that can be processed by the
CNN model. For example, we can rst split our univariate time series data into input/output
samples with four steps as input and one as output. Each sample can then be split into two
sub-samples, each with two time steps. The CNN can interpret each subsequence of two time
steps and provide a time series of interpretations of the subsequences to the LSTM model
to process as input. We can parameterize this and de ne the number of subsequences as
n
seq and the number of time steps per subsequence as n
steps. The input data can then be
reshaped to have the required structure: [samples, subsequences, timesteps, features].
For example:

In [ ]:
# choose a number of time steps
n_steps = 4
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, n_steps, n_features))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, TimeDistributed, Conv1D, MaxPooling1D




In [ ]:
"""# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, rows, columns, features]
n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, 1, n_steps, n_features))"""

'# choose a number of time steps\nn_steps = 3\n# split into samples\nX, y = split_sequence(raw_seq, n_steps)\n# reshape from [samples, timesteps] into [samples, timesteps, rows, columns, features]\nn_features = 1\nn_seq = 2\nn_steps = 2\nX = X.reshape((X.shape[0], n_seq, 1, n_steps, n_features))'

### ConvLSTM

A type of LSTM related to the CNN-LSTM is the ConvLSTM, where the convolutional reading
of input is built directly into each LSTM unit. The ConvLSTM was developed for reading
two-dimensional spatial-temporal data, but can be adapted for use with univariate time series
forecasting. The layer expects input as a sequence of two-dimensional images, therefore the
shape of input data must be: [samples, timesteps, rows, columns, features].
For our purposes, we can split each sample into subsequences where timesteps will become
the number of subsequences, or n
each subsequence, or n
seq, and columns will be the number of time steps for
steps. The number of rows is xed at 1 as we are working with
one-dimensional data. We can now reshape the prepared samples into the required structure

In [ ]:
# choose a number of time steps
n_steps = 4
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, rows, columns, features]
n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, 1, n_steps, n_features))

In [ ]:
from keras.layers import Flatten
from keras.layers import ConvLSTM2D
 
# define the input cnnlstm model
"""model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(n_seq,
1, n_steps, n_features)))
model.add(Flatten())"""

"model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(n_seq,\n1, n_steps, n_features)))\nmodel.add(Flatten())"

## UNIVARIATE FORECASTING

Revue de littérature, on va parler de ceci:

The goal of the study was to clearly demonstrate the capability of a suite of di erent machine
learning methods as compared to classical time series forecasting methods on a very large and
diverse collection of univariate time series forecasting problems. The study was a response to
the increasing number of papers and claims that machine learning and deep learning methods
o er superior results for time series forecasting with little objective evidence.
Literally hundreds of papers propose new ML algorithms, suggesting methodological
advances and accuracy improvements. Yet, limited objective evidence is available
regarding their relative performance as a standard forecasting tool.
Statistical and Machine Learning forecasting methods: Concerns and ways forward, 2018.
The authors clearly lay out three issues with the ood of claims; they are:
ˆ
ˆ
ˆ
Their conclusions are based on a few, or even a single time series, raising questions about
the statistical signi cance of the results and their generalization.
The methods are evaluated for short-term forecasting horizons, often one-step-ahead, not
considering medium and long-term ones.
No benchmarks are used to compare the accuracy of ML methods versus alternative ones.
As a response, the study includes eight classical methods and 10 machine learning methods
evaluated using one-step and multiple-step forecasts across a collection of 1,045 monthly time
series. Although not de nitive, the results are intended to be objective and robust.
10.3 Time Series Datasets
The time series datasets used in the study were drawn from the time series datasets used in the
M3-Competition. The M3-Competition was the third in a series of competitions that sought to
discover exactly what algorithms perform well in practice on real time series forecasting problems.
The results of the competition were published in the 2000 paper titled The M3-Competition:
Results, Conclusions and Implications. The datasets used in the competition were drawn from a
wide range of industries and had a range of di erent time intervals, from hourly to annual.
10.4. Time Series Forecasting Methods
165
The 3003 series of the M3-Competition were selected on a quota basis to include
various types of time series data (micro, industry, macro, etc.) and di erent time
intervals between successive observations (yearly, quarterly, etc.).
The M3-Competition: Results, Conclusions and Implications, 2000.
The table below, taken from the paper, provides a summary of the 3,003 datasets used in
the competition.
Figure 10.1: Table of Datasets, Industry and Time Interval Used in the M3-Competition. Taken
from The M3-Competition: Results, Conclusions and Implications.
The nding of the competition was that simpler time series forecasting methods outperform
more sophisticated methods, including neural network models.
This study, the previous two M-Competitions and many other empirical studies have
proven, beyond the slightest doubt, that elaborate theoretical constructs or more
sophisticated methods do not necessarily improve post-sample forecasting accuracy,
over simple methods, although they can better t a statistical model to the available
historical data.
The M3-Competition: Results, Conclusions and Implications, 2000.
The more recent study that we are reviewing in this tutorial that evaluate machine learning
methods selected a subset of 1,045 time series with a monthly interval from those used in the
M3 competition.
... evaluate such performance across multiple forecasting horizons using a large
subset of 1045 monthly time series used in the M3 Competition.
Statistical and Machine Learning forecasting methods: Concerns and ways forward, 2018
 Time Series Forecasting Methods
 The study evaluates the performance of eight classical (or simpler) methods and 10 machine
 learning methods.
 ... of eight traditional statistical methods and eight popular ML ones, [...], plus two
 more that have become popular during recent years.
10.4. Time Series Forecasting Methods
 166
 Statistical and Machine Learning forecasting methods: Concerns and ways forward, 2018.
 The eight classical methods evaluated were as follows:
 ˆ
 ˆ
 ˆ
 ˆ
 ˆ
 ˆ
 ˆ
 ˆ
 Naive 2, which is actually a random walk model adjusted for season.
 Simple Exponential Smoothing.
 Holt.
 Damped exponential smoothing.
 Average of SES, Holt, and Damped.
 Theta method.
 ARIMA, automatic.
 ETS, automatic.
 A total of eight machine learning methods were used in an e ort to reproduce and compare
 to results presented in the 2010 paper An Empirical Comparison of Machine Learning Models
 for Time Series Forecasting. They were:
 ˆ
 ˆ
 ˆ
 ˆ
 ˆ
 ˆ
 ˆ
 ˆ
 Multilayer Perceptron (MLP).
 Bayesian Neural Network (BNN).
 Radial Basis Functions (RBF).
 Generalized Regression Neural Networks (GRNN), also called kernel regression.
 k-Nearest Neighbor regression (KNN).
 CART regression trees (CART).
 Support Vector Regression (SVR).
 Gaussian Processes (GP).
 An additional two modern neural network algorithms were also added to the list given the
 recent rise in their adoption; they were:
 ˆ
 ˆ
 Recurrent Neural Network (RNN).
 Long Short-Term Memory (LSTM).
10.5. Data Preparation

Data Preparation
 A careful data preparation methodology was used, again, based on the methodology described
 in the 2010 paper An Empirical Comparison of Machine Learning Models for Time Series
 Forecasting. In that paper, each time series was adjusted using a power transform, deseasonalized
 and detrended.
 ... before computing the 18 forecasts, they preprocessed the series in order to achieve
 stationarity in their mean and variance. This was done using the log transformation,
 then deseasonalization and nally scaling, while rst di erences were also considered
 for removing the component of trend.
 Statistical and Machine Learning forecasting methods: Concerns and ways forward, 2018.
 Inspired by these operations, variations of ve di erent data transforms were applied for an
 MLP for one-step forecasting and their results were compared. The ve transforms were:
 ˆ
 ˆ
 ˆ
 ˆ
 ˆ
 Original data.
 Box-Cox Power Transform.
 Deseasonalizing the data.
 Detrending the data.
 All three transforms (power, deseasonalize, detrend).
 Generally, it was found that the best approach was to apply a power transform and desea
sonalize the data, and perhaps detrend the series as well.
 The best combination according to sMAPE is number 7 (Box-Cox transformation,
 deseasonalization) while the best one according to MASE is number 10 (Box-Cox
 transformation, deseasonalization and detrending)
 Statistical and Machine Learning forecasting methods: Concerns and ways forward, 2018.
 10.6 One-step Forecasting Results
 All models were evaluated using one-step time series forecasting. Speci cally, the last 18 time
 steps were used as a test set, and models were t on all remaining observations. A separate
 one-step forecast was made for each of the 18 observations in the test set, presumably using a
 walk-forward validation method where true observations were used as input in order to make
 each forecast.
 The forecasting model was developed using the rst n- 18 observations, where n
 is the length of the series. Then, 18 forecasts were produced and their accuracy
 was evaluated compared to the actual values not used in developing the forecasting
 model.
10.6. One-step Forecasting Results
 168
 Statistical and Machine Learning forecasting methods: Concerns and ways forward, 2018
 Reviewing the results, the MLP and BNN were found to achieve the best performance from
 all of the machine learning methods.
 The results [...] show that MLP and BNN outperform the remaining ML methods.
 Statistical and Machine Learning forecasting methods: Concerns and ways forward, 2018.
 An interesting result was that RNNs and LSTMs were found to perform poorly. This
 con rms a earlier nding in the application of LSTMs to univariate time series forecast reported
 by Gers, et al. in 2001 titled Applying LSTM to Time Series Predictable through Time-Window
 Approaches. In that study, they also reported that LSTMs are easily outperformed by simpler
 methods and may not be suited to simple autoregressive-type univariate time series forecasting
 problems.
 It should be noted that RNN is among the less accurate ML methods, demonstrating
 that research progress does not necessarily guarantee improvements in forecasting
 performance. This conclusion also applies in the performance of LSTM, another
 popular and more advanced ML method, which does not enhance forecasting accuracy
 too.
 Statistical and Machine Learning forecasting methods: Concerns and ways forward, 2018.
 Comparing the performance of all methods, it was found that the machine learning methods
 were all out-performed by simple classical methods, where ETS and ARIMA models performed
 the best overall. This nding con rms the results from previous similar studies and competitions.
 Figure 10.2: Bar Chart Comparing Model Performance (sMAPE) for One-step Forecasts. Taken
 from Statistical and Machine Learning forecasting methods: Concerns and ways forward.




### Simple methods for univariate forecasting

In [ ]:
# define model
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(n_seq,
1, n_steps, n_features)))
model.add(Flatten())
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=500, verbose=0)
# demonstrate prediction
x_input = array([147995,  99996, 137198])
x_input = x_input.reshape((1, n_seq, 1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


[[125350.42]]


In [ ]:
# define univariate time series
series = array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
print(series.shape)

# transform to a supervised learning problem
X, y = split_sequence(series, 3)
print(X.shape, y.shape)

# show each sample
for i in range(len(X)):
    print(X[i], y[i])

(10,)
(7, 3) (7,)
[1 2 3] 4
[2 3 4] 5
[3 4 5] 6
[4 5 6] 7
[5 6 7] 8
[6 7 8] 9
[7 8 9] 10


### 3D Data Preparation Basics

In [ ]:
# transform univariate 2d to 3d
from numpy import array
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)
# define univariate time series
series = array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
print(series.shape)
# transform to a supervised learning problem
X, y = split_sequence(series, 3)
print(X.shape, y.shape)
# transform input from [samples, features] to [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))
print(X.shape)

(10,)
(7, 3) (7,)
(7, 3, 1)


#### Data preparation Example

In [ ]:
import pandas as pd     
import matplotlib as plt   

# example of defining a dataset
from numpy import array
# define the dataset
data = list()
n = 5000
for i in range(n):
    data.append([i+1, (i+1)*10])
data = array(data)
print(data[:5, :])
print(data.shape)

[[ 1 10]
 [ 2 20]
 [ 3 30]
 [ 4 40]
 [ 5 50]]
(5000, 2)


In [ ]:
#drop the time column
# drop time
data = data[:, 1]
print(data.shape)



(5000,)


In [ ]:
# drop time
data = data[:, 1]
# split into samples (e.g. 5000/200 = 25)
samples = list()
length = 200
# step over the 5,000 in jumps of 200
for i in range(0,n,length):
    # grab from i to i + 200
    sample = data[i:i+length]
    samples.append(sample)
print(len(samples))

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
# example of creating an array of subsequence
from numpy import array
# define the dataset
data = list()
n = 5000
for i in range(n):
    data.append([i+1, (i+1)*10])
    
data = array(data)

# drop time
data = data[:, 1]
# split into samples (e.g. 5000/200 = 25)
samples = list()
length = 200
# step over the 5,000 in jumps of 200
for i in range(0,n,length):
    # grab from i to i + 200
    sample = data[i:i+length]
    samples.append(sample)
# convert list of arrays into 2d array
data = array(samples)
print(data.shape)

(25, 200)


In [ ]:
# reshape into [samples, timesteps, features]
data = data.reshape((len(samples), length, 1))
print(data.shape)

(25, 200, 1)


### Simple methods for univariate forecasting